# From Arturo Filastò

In [1]:
#!pip install boto3
#!pip install lz4framed
import os
import io
import yaml
import json
from datetime import datetime, timedelta

import pandas as pd
import psycopg2
import boto3

import requests
from tqdm import tqdm

from dateutil.parser import parse as parse_date
from urllib.parse import urlencode, quote, urlparse

import requests
#import lz4framed
from urllib.parse import urljoin
import gzip
import subprocess
import tarfile

In [2]:
from botocore import UNSIGNED
from botocore.config import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [3]:
CANNED_BASE_URL = 'http://s3.amazonaws.com/ooni-data/canned/'
def load_canned_index(bucket_date):
    r = requests.get(
            urljoin(CANNED_BASE_URL, '{}/index.json.gz'.format(bucket_date))
        )
    blob = r.content
    return gzip.decompress(blob).split(b'\n')

In [4]:
def decompress_tar(filename):
    tar_data = b''
    p = subprocess.Popen(["lz4", "-d"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    r = requests.get(urljoin(CANNED_BASE_URL, filename))
    stdout_data, _ = p.communicate(input=r.content)
    return tarfile.open(mode="r:", fileobj=io.BytesIO(stdout_data))
#     COUNTRY_CODE = "RU"

canned_index = load_canned_index('2020-07-01')
  ## XXX need to filter by test_name
can = json.loads(canned_index[3])
tarfd = decompress_tar(can['filename'])
print(tarfd.getmembers())

[<TarInfo '2020-07-01/20200701T184544Z-BR-AS27699-web_connectivity-20200701T184551Z_AS27699_9SKyvqgf8GlMys15QwKaMB6NxZRqSVW4Je03ghocv9Lf195iNt-0.2.0-probe.json' at 0x7fe851a73d50>, <TarInfo '2020-07-01/20200701T185245Z-ZA-AS328453-web_connectivity-20200701T185246Z_AS328453_xKFIDJUHwTPzJDLoEzdqfo4ZaqCunNvwq7MI0oBOrxVs7BE2dz-0.2.0-probe.json' at 0x7fe851a73ef0>, <TarInfo '2020-07-01/20200701T185348Z-BR-AS27699-web_connectivity-20200701T185352Z_AS27699_hHdbyxJNkr3q2z4KYIY2l0TtHbq2MPv6KBvj2fMDCZLclMZNhC-0.2.0-probe.json' at 0x7fe851bca120>, <TarInfo '2020-07-01/20200701T185552Z-MA-AS36903-web_connectivity-20200701T185553Z_AS36903_MQP35vFn698w0uTaBW5UHdsz70GDRy0vsBkW5LmQ9oyDmy3hZw-0.2.0-probe.json' at 0x7fe851bca2c0>, <TarInfo '2020-07-01/20200701T185623Z-BR-AS1916-web_connectivity-20200701T185623Z_AS1916_uDhSU5ySYCHAdAE6VQzm8fB8mLMhjmG8zFfs7y36K1qyjsaADx-0.2.0-probe.json' at 0x7fe851bca460>, <TarInfo '2020-07-01/20200701T185801Z-US-AS20115-web_connectivity-20200701T185804Z_AS20115_TCjlkThs

# Russia Data January to July- Sarah#

So I just pulled in params from the API documentation from the URL for the json data that I was using earlier. I created a params list and pulled from a request and printed it. I appended the info to a list and in this example, I put it into a dataframe. 

We can add params from the documentation and mess with this data fairly easily. 
https://api.ooni.io/api/ 

https://api.ooni.io/api/v1/aggregation?since=2020-03-01&until=2020-07-31&axis_x=probe_cc&axis_y=measurement_start_day&test_name=web_connectivity&probe_cc=RU&input=https://twitter.com/ <-example url I built this off of.

Also, I would like to clean up this notebook by making it more dynamic if you can. I would like to have the target url and the params listed once and use variables in place of "2020-01-01", if we could store that in a variable like start_date and end_date for "2020-06-30". I would like it to update the params but not list them out for every country if you know what I mean so it looks cleaner and we can have "probe_cc":"country_code", and just put country_code = "RU" so when we run the block of cells for Russia it updates the url but it is essentially more dynamic and less code. Try not to drop the country code just yet. I pulled for 5 total countries out of caution. 

In [57]:
target_url = "https://api.ooni.io/api/v1/aggregation"

params = {
    "since": "2020-01-01",  
    "until": "2020-06-30",
    "axis_x": "probe_cc",
    "axis_y": "measurement_start_day",
    "test_name": "web_connectivity",
    "probe_cc":"RU",
    "input": "https://instagram.com/"
}


In [58]:
response = requests.get(target_url, params)
russia_data = response.json()


In [59]:
russia_list = []

for item in russia_data["result"]:
    russia_list.append(item)


In [60]:
import pandas as pd
russia_df = pd.DataFrame(russia_list)

russia_df['failure_count'].sum()
russia_df

,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_start_day,probe_cc
0,11910,3305,4,76997,2020-01-02,RU
1,12876,3638,4,91732,2020-01-03,RU
2,9340,2769,4,81547,2020-01-04,RU
3,10821,2964,7,89604,2020-01-05,RU
4,10185,2979,12,85468,2020-01-06,RU
...,...,...,...,...,...,...
168,17368,3140,181,81209,2020-06-26,RU
169,17500,3254,78,82900,2020-06-27,RU
170,17559,3172,179,81203,2020-06-28,RU
171,18148,3128,173,83306,2020-06-29,RU


# USA Data from January to June

In [50]:
target_url = "https://api.ooni.io/api/v1/aggregation"

params = {
    "since": "2020-01-01",  
    "until": "2020-06-30",
    "axis_x": "probe_cc",
    "axis_y": "measurement_start_day",
    "test_name": "web_connectivity",
    "probe_cc":"US",
    "input": "https://twitter.com/"
}

In [51]:
response = requests.get(target_url, params)
usa_data = response.json()

In [36]:
usa_list = []

for item in usa_data["result"]:
    usa_list.append(item)

In [53]:
usa_df = pd.DataFrame(usa_list)

usa_df

,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_start_day,probe_cc
0,5089,0,83,29809,2020-01-02,US
1,6434,0,132,40102,2020-01-03,US
2,6269,0,87,27692,2020-01-04,US
3,5486,0,105,27346,2020-01-05,US
4,5449,0,58,31773,2020-01-06,US
...,...,...,...,...,...,...
168,5011,0,2294,31661,2020-06-26,US
169,5125,0,2438,37194,2020-06-27,US
170,4893,0,2292,28116,2020-06-28,US
171,5834,0,2303,42844,2020-06-29,US


# China Data from January to June

In [38]:
target_url = "https://api.ooni.io/api/v1/aggregation"

params = {
    "since": "2020-01-01",  
    "until": "2020-06-30",
    "axis_x": "probe_cc",
    "axis_y": "measurement_start_day",
    "test_name": "web_connectivity",
    "probe_cc":"CN",
    "input": "https://twitter.com/"
}

In [39]:
response = requests.get(target_url, params)
china_data = response.json()

In [40]:
china_list = []

for item in china_data["result"]:
    china_list.append(item)

In [54]:
china_df = pd.DataFrame(china_list)

china_df

,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_start_day,probe_cc
0,24,0,12,92,2020-01-02,CN
1,16,0,4,52,2020-01-03,CN
2,21,0,11,109,2020-01-04,CN
3,11,0,4,80,2020-01-05,CN
4,22,0,5,100,2020-01-06,CN
...,...,...,...,...,...,...
168,3,0,0,4,2020-06-26,CN
169,22,0,0,60,2020-06-27,CN
170,9,0,0,32,2020-06-28,CN
171,13,0,0,53,2020-06-29,CN


# Venezuela data from January to June

In [42]:
target_url = "https://api.ooni.io/api/v1/aggregation"

params = {
    "since": "2020-01-01",  
    "until": "2020-06-30",
    "axis_x": "probe_cc",
    "axis_y": "measurement_start_day",
    "test_name": "web_connectivity",
    "probe_cc":"VE",
    "input": "https://twitter.com/"
}

In [43]:
response = requests.get(target_url, params)
venezuela_data = response.json()

In [44]:
venezuela_list = []

for item in venezuela_data["result"]:
    venezuela_list.append(item)

In [45]:
# venezuela_list

# Morocco from January to June

In [46]:
target_url = "https://api.ooni.io/api/v1/aggregation"

params = {
    "since": "2020-01-01",  
    "until": "2020-06-30",
    "axis_x": "probe_cc",
    "axis_y": "measurement_start_day",
    "test_name": "web_connectivity",
    "probe_cc":"MA",
    "input": "https://twitter.com/"
}

In [47]:
response = requests.get(target_url, params)
morocco_data = response.json()

In [48]:
morocco_list = []

for item in morocco_data["result"]:
    morocco_list.append(item)

In [49]:
morocco_list

[{'anomaly_count': 4,
  'confirmed_count': 0,
  'failure_count': 3,
  'measurement_count': 56,
  'measurement_start_day': '2020-01-07',
  'probe_cc': 'MA'},
 {'anomaly_count': 0,
  'confirmed_count': 0,
  'failure_count': 0,
  'measurement_count': 34,
  'measurement_start_day': '2020-01-09',
  'probe_cc': 'MA'},
 {'anomaly_count': 1,
  'confirmed_count': 0,
  'failure_count': 0,
  'measurement_count': 13,
  'measurement_start_day': '2020-01-10',
  'probe_cc': 'MA'},
 {'anomaly_count': 1,
  'confirmed_count': 0,
  'failure_count': 0,
  'measurement_count': 57,
  'measurement_start_day': '2020-01-12',
  'probe_cc': 'MA'},
 {'anomaly_count': 0,
  'confirmed_count': 0,
  'failure_count': 0,
  'measurement_count': 23,
  'measurement_start_day': '2020-01-13',
  'probe_cc': 'MA'},
 {'anomaly_count': 1,
  'confirmed_count': 0,
  'failure_count': 2,
  'measurement_count': 21,
  'measurement_start_day': '2020-01-14',
  'probe_cc': 'MA'},
 {'anomaly_count': 1,
  'confirmed_count': 0,
  'failure_c